In [1]:
!pip install -U -q keras-nlp
!pip install --upgrade keras
!pip install kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [3]:
import os
from google.colab import userdata
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [4]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [5]:
import os
os.environ["KERAS_BACKEND"] = "jax"

In [6]:
%%capture
import kagglehub
path = kagglehub.model_download("metaresearch/llama-3.2/transformers/3b-instruct")

In [7]:
print("Path to model files:", path)

Path to model files: /root/.cache/kagglehub/models/metaresearch/llama-3.2/transformers/3b-instruct/1


In [8]:
!pip install opendatasets

In [9]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/mohamedkhamis/llama3-1-dependencies")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: syedosamaalishah092
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/mohamedkhamis/llama3-1-dependencies


100%|██████████| 2.78G/2.78G [02:49<00:00, 17.6MB/s]


In [12]:
!pip install -q -U bitsandbytes --no-index --find-links ../content/llama3-1-dependencies/dependencies/
!pip install -q -U transformers --no-index --find-links ../content/llama3-1-dependencies/dependencies/
!pip install -q -U tokenizers --no-index --find-links ../content/llama3-1-dependencies/dependencies/
!pip install -q -U peft --no-index --find-links ../content/llama3-1-dependencies/dependencies/

In [11]:
!pip install -U trl

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is th

In [13]:
import numpy as np
import pandas as pd


import os
for dirname, _, filenames in os.walk('/content'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/.config/.last_opt_in_prompt.yaml
/content/.config/config_sentinel
/content/.config/gce
/content/.config/default_configs.db
/content/.config/active_config
/content/.config/.last_update_check.json
/content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
/content/.config/.last_survey_prompt.yaml
/content/.config/configurations/config_default
/content/.config/logs/2024.09.25/18.23.58.041880.log
/content/.config/logs/2024.09.25/18.23.16.379542.log
/content/.config/logs/2024.09.25/18.24.13.969738.log
/content/.config/logs/2024.09.25/18.24.13.190037.log
/content/.config/logs/2024.09.25/18.23.56.396629.log
/content/.config/logs/2024.09.25/18.23.40.958965.log
/content/llama3-1-dependencies/dependencies/nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl
/content/llama3-1-dependencies/dependencies/fsspec-2024.6.1-py3-none-any.whl
/content/llama3-1-dependencies/dependencies/nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl
/content/llama3-1-

In [14]:
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    DataCollatorWithPadding,
    LlamaForSequenceClassification,
    LlamaTokenizerFast,
    PreTrainedTokenizerBase,
    EvalPrediction,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftModel
from sklearn.metrics import log_loss, accuracy_score

In [15]:
@dataclass
class Config:
    output_dir: str = "output"
    checkpoint: str = "/root/.cache/kagglehub/models/metaresearch/llama-3.2/transformers/3b-instruct/1"
    max_length: int = 2048
    n_splits: int = 5
    fold_idx: int = 0
    optim_type: str = "adamw_8bit"
    per_device_train_batch_size: int = 4
    gradient_accumulation_steps: int = 4
    per_device_eval_batch_size: int = 8
    n_epochs: int = 1
    freeze_layers: int = 16
    lr: float = 2e-4
    warmup_steps: int = 20
    lora_r: int = 4
    lora_alpha: float = lora_r * 2
    lora_dropout: float = 0.05
    lora_bias: str = "none"

config = Config()

In [16]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    target_modules=["q_proj", "k_proj", "v_proj"],
    layers_to_transform=[i for i in range(32) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)

In [17]:
tokenizer = LlamaTokenizerFast.from_pretrained(config.checkpoint)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.padding_side = "right"

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizerFast'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [18]:
model = LlamaForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /root/.cache/kagglehub/models/metaresearch/llama-3.2/transformers/3b-instruct/1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
              (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
              (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
              (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
              (rotary_emb): LlamaRotaryEmbedding()
            )
            (mlp): LlamaMLP(
              (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
              (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
              (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
              (act_fn): SiLU()
        

In [20]:
model.print_trainable_parameters()

trainable params: 697,344 || all params: 3,213,456,384 || trainable%: 0.0217


In [21]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

In [26]:
import pandas as pd
df = pd.read_csv('/content/Dataset/train (1).csv')
df = df.dropna()
df = df.drop_duplicates(subset=['response_a', 'response_b'], keep=False)
df["len"] = df["prompt"].apply(len) + df["response_a"].apply(len) + df["response_b"].apply(len)
df = df.sort_values(by=['len'])
df

<ipython-input-26-d8e2a73bf9ec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["len"] = df["prompt"].apply(len) + df["response_a"].apply(len) + df["response_b"].apply(len)


,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,len
10318,766523855,claude-2.1,claude-1,"[""Say \""hi\""""]","[""Hi""]","[""Hi""]",0,0,1,26
10294,765312542,wizardlm-70b,vicuna-33b,"[""cat sat on a ""]","[""mat""]","[""mat""]",0,0,1,31
581,43245275,claude-1,pplx-7b-online,"[""what is 1+1?""]","[""1 + 1 = 2""]","[""""]",1,0,0,33
42450,3160426362,dolly-v2-12b,claude-instant-1,"[""Write a single dot""]","["".\\""]","["".""]",0,1,0,34
48894,3651614122,claude-1,gemini-pro,"[""What is 4-2?""]","[""4 - 2 = 2""]","[""2""]",0,0,1,34
...,...,...,...,...,...,...,...,...,...,...
46908,3501226594,gpt-4-0314,claude-2.1,"[""accustom \u548c fit adpat acclimatize \u7684...","[""\""Accustom\"", \""fit\"", \""adapt\"", \u548c \""a...","[""accustom, fit, adapt, acclimatize \u8fd9\u4e...",0,0,1,59890
32650,2425787535,chatglm3-6b,claude-1,"[""so i was sleeping but i woke up to seeing a ...","[""It sounds like you had a unique and interest...","[""Oh, that sounds like an interesting experien...",0,1,0,60604
51836,3874210883,openhermes-2.5-mistral-7b,claude-1,"[""The process of leaching in land pollution re...","[""d. The release of pollutants into the soil, ...","[""The correct answer is d. The release of poll...",0,0,1,62941
40208,2995198009,gpt-4-0125-preview,deepseek-llm-67b-chat,"[""Let's play a text based adventure game, it's...","[""You find yourself standing at the edge of th...","[""Sure, I can play the sassy, sarcastic compan...",1,0,0,72079


In [27]:
ds = Dataset.from_pandas(df)
ds = ds.select(torch.arange(1000))

In [28]:
class CustomTokenizer:
    def __init__(
        self,
        tokenizer: PreTrainedTokenizerBase,
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, batch: dict) -> dict:
        prompt = ["Which is the better response for the prompt? response_a or response_b or tie? \n'n give score for each lable \n\n <prompt>: " + self.process_text(t) for t in batch["prompt"]]
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        labels=[]
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:
                label = 0
            elif b_win:
                label = 1
            else:
                label = 2
            labels.append(label)
        return {**tokenized, "labels": labels}

    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))

In [29]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)
ds = ds.map(encode, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [30]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    # Check for NaNs in predictions and labels
    if np.isnan(probs).any() or np.isnan(labels).any():
        raise ValueError("NaN values found in predictions or labels")

    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}

In [31]:
folds = [
        (
            [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
            [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
        )
        for fold_idx in range(config.n_splits)
    ]

In [32]:
from trl import SFTTrainer, SFTConfig
sft_config = SFTConfig(
    output_dir="output",
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=1000,
    save_strategy="epoch",
    save_steps=100,
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
    packing=True,
    dataset_text_field="text",
    max_seq_length=config.max_length,
)

In [33]:
trainer = SFTTrainer(
        model,
        train_dataset=ds,
        args=sft_config
    )

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [34]:
for fold_idx in range(config.n_splits):

    train_idx, eval_idx = folds[fold_idx]

    train_data = ds.select(train_idx).sort("len")
    val_data = ds.select(eval_idx).sort("len")

    batch_size = 200
    num_batches = len(train_data) // batch_size + (1 if len(train_data) % batch_size != 0 else 0)

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, len(train_data))
        ds_temp = train_data.select(range(start_idx, end_idx))

        trainer.train_dataset = ds_temp

        print(f"Training batch {batch_idx + 1}/{num_batches} on fold {fold_idx + 1}/{config.n_splits}...")

        trainer.train()

        trainer.save_model(f"model_fold_{fold_idx}_batch{batch_idx}")


    trainer.eval_dataset = val_data

    print(f"Validating on fold {fold_idx + 1}/{config.n_splits}...")
    eval_results = trainer.evaluate()

    # Save metrics if needed
    print(f"Evaluation results for fold {fold_idx + 1}: {eval_results}")


Training batch 1/4 on fold 1/5...


Step,Training Loss


Training batch 2/4 on fold 1/5...


Step,Training Loss


Training batch 3/4 on fold 1/5...


...

Training batch 3/4 on fold 5/5...


Step,Training Loss


Training batch 4/4 on fold 5/5...


Step,Training Loss


Validating on fold 5/5...
Evaluation results for fold 5: {'eval_loss': 0.9020636081695557, 'eval_runtime': 4.4029, 'eval_samples_per_second': 45.425, 'eval_steps_per_second': 5.678, 'epoch': 0.96}
